#### 1. Importing Necessary Libraries

In [ ]:
pip install PyPDF2
pip install Pillow
pip install numpy
pip install pandas
pip install PyMuPDF
pip install pytesseract
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' #adjust as needed based on path: for windows
pytesseract.pytesseract.tesseract_cmd = '/usr/local/bin/tesseract' # for mac
pip install IPython
pip install fitz
pip install spacy

In [ ]:
#import libraries
import PyPDF2
from PIL import Image
import numpy as np
import pandas as pd
import os
import re
import fitz 
import pytesseract
import spacy
from spacy.training.example import Example
import random
from collections import defaultdict
import ast

#### 2. Choosing Merged PDF Document

In [ ]:
#choose the merged PDF from your folder
import tkinter as tk
from tkinter import filedialog

# Create a Tkinter root window
root = tk.Tk()
root.withdraw()  # Hide the root window

# Ask the user to select a file
pdf_path = filedialog.askopenfilename(title="Select PDF file")

# Check if the user selected a file
if pdf_path:
    print("Selected PDF file:", pdf_path)
else:
    print("No file selected.")

In [ ]:
# Extract the directory path from the pdf_path
pdf_directory = os.path.dirname(pdf_path)

# Define the paths for the new folders
pdf_folder = os.path.join(pdf_directory, "pdf_folder")

# Create the new folders
if not os.path.exists(pdf_folder):
    os.makedirs(pdf_folder)

#### 3. Splitting the individual PDFs from the Merged PDF

In [ ]:
def find_firstpage(page_text):
    # Regular expression pattern to find "Page: 1 of X" footer text
    pattern = r"Page:\s*1\s*of\s*(\d+)"
    match = re.search(pattern, page_text)
    if match:
        return int(match.group(1))
    else:
        return None

In [ ]:
def split_pdf(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        total_pages = len(reader.pages)
        current_page = 0
        section_counter = 1
        
        # Create the pdf folder if it doesn't exist
        if not os.path.exists(pdf_folder):
            os.makedirs(pdf_folder)
        
        while current_page < total_pages:
            writer = PyPDF2.PdfWriter()
            start_page = current_page + 1
            
            # Extract text from the first page of the section
            first_page = reader.pages[current_page]
            page_text = first_page.extract_text()
            footer_text = find_firstpage(page_text)
            
            if footer_text:
                # If footer text is found, split the section based on the number of pages indicated
                for _ in range(footer_text):
                    if current_page < total_pages:
                        writer.add_page(reader.pages[current_page])
                        current_page += 1
            else:
                # If no footer text is found, treat the section as a single-page section
                writer.add_page(first_page)
                current_page += 1
                
            pdf_name = f"pdf{section_counter}.pdf"
            pdf_pdf_path = os.path.join(pdf_folder, pdf_name)
            #pdf_pdf_path = os.path.join(pdf_folder, pdf_path.replace('.pdf', f'_part{start_page}_to_{current_page}.pdf').split('/')[-1])
            with open(pdf_pdf_path, 'wb') as out:
                writer.write(out)
                
            section_counter += 1

In [ ]:
split_pdf(pdf_path)

#### 4. Rotating pages for each PDF in the correct orientation

In [ ]:
def get_page_orientation(page):
    # Extract text elements
    text_elements = page.get_text("dict")["blocks"]
    
    # Analyze text block distribution
    num_vertical_blocks = 0
    num_horizontal_blocks = 0
    for block in text_elements:
        bbox = block["bbox"]
        width = bbox[2] - bbox[0]
        height = bbox[3] - bbox[1]
        if width > height:
            num_horizontal_blocks += 1
        else:
            num_vertical_blocks += 1
    
    # Determine orientation based on block distribution
    if num_horizontal_blocks > num_vertical_blocks:
        return "Portrait"
    else:
        return "Landscape"

def rotate_page_to_portrait(pdf_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    rotated_pages = []

    # Iterate through each page
    for page_number in range(len(pdf_document)):
        page = pdf_document.load_page(page_number)
        
        # Get the page orientation
        orientation = get_page_orientation(page)
        
        # Check if the page is not in portrait orientation
        if orientation == "Portrait":
            # Rotate the page to portrait (90 degrees)
            page.set_rotation(0)
        elif orientation == "Landscape":
            # Rotate the page to portrait (0 degrees)
            page.set_rotation(90)
            rotated_pages.append(page_number + 1)  # Store the page number
            
    # Save the modified PDF with rotated pages to a temporary file
    temp_pdf_path = os.path.splitext(pdf_path)[0] + "_temp.pdf"
    pdf_document.save(temp_pdf_path)

    # Close the PDF document
    pdf_document.close()

    # Replace the original PDF with the rotated one
    os.replace(temp_pdf_path, pdf_path)

    return rotated_pages

In [ ]:
#rotate all PDFs to the correct orientation 
for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):  # Check if the file is a PDF
            pdf_file = os.path.join(pdf_folder, filename)
            rotate_page_to_portrait(pdf_file)

#### 5. Deleting pages in PDF except the page containing Composition Table

In [ ]:
def find_word_in_pdf(pdf_path):
    with open(pdf_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        num_pages = len(reader.pages)

        composition_found = False

        composition_word = 'composition'
        composition_sublot = 'composition sublot a and b'
        formulation_word = 'formulation'
        composition_solvent = 'solvent composition'

        # Loop through each page to find "composition" or "formulation"
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text = page.extract_text().lower()
            
            # Check if "composition" is present
            if composition_word in text:
                composition_found = True
                if composition_solvent in text:
                    composition_count = text.count(composition_word)
                    if composition_count > 1:
                        return page_num + 1
                elif composition_sublot in text:
                    continue
                else:
                    return page_num + 1
                    
        if not composition_found:
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                text = page.extract_text().lower()

                if formulation_word in text:
                    return page_num + 1

        return None

In [ ]:
def delete_pages(pdf_path, page_to_keep):
    temp_path = 'temp.pdf'  # Temporary file path

    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        writer = PyPDF2.PdfWriter()

        for page_num in range(len(reader.pages)):
            if page_num + 1 == page_to_keep:
                page = reader.pages[page_num]
                writer.add_page(page)

        with open(temp_path, 'wb') as output_file:
            writer.write(output_file)

    # Replace the original file with the temporary file
    os.replace(temp_path, pdf_path)

In [ ]:
# Iterate through each PDF file in the pdf_folder
def extract_index(filename):
    # Use regular expression to find the numerical part of the filename
    match = re.search(r'\d+', filename)
    if match:
        return int(match.group())
    else:
        return -1 

# Iterate through each PDF file in the pdf_folder
for filename in sorted(os.listdir(pdf_folder), key=extract_index):
    if filename.endswith(".pdf"): 
        pdf_file = os.path.join(pdf_folder, filename)
        
        page_number = find_word_in_pdf(pdf_file)
        print(f"Filename: {filename}, Page number: {page_number}")
        
        if page_number is not None:
            delete_pages(pdf_file, page_number)
        
        elif page_number is None:
            os.remove(pdf_file)

#### 6. Extracting Batch Number + Composition Table

In [ ]:
def extract_batchnumber(pdf_path):
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text()

    # Use regular expression to find "batch number:" followed by any characters (.*)
    match = re.search(r'number:\s*(.*)', text, re.IGNORECASE)

    if match:
        batch_number = match.group(1).strip()
        # Capitalize the words in the batch number
        batch_number = batch_number.upper()
        return batch_number
    else:
        return None

In [ ]:
from img2table.document import PDF
from img2table.ocr import TesseractOCR
from IPython.display import display_html

# Set environment path in the script
os.environ['PATH'] += os.pathsep + '/opt/homebrew/bin' 
# set correct path #/opt/homebrew/bin is typical for Unix-like systems (macOS and Linux) where Homebrew, a package manager, is installed

# Windows-style paths: For example, if you've installed a program (like Tesseract OCR) in the Program Files directory, you would update your PATH like this
# os.environ['PATH'] += os.pathsep + 'C:\\Program Files\\Tesseract-OCR'

# Instantiate OCR
ocr = TesseractOCR(n_threads=1, lang="eng")

def extract_pdfcompositiontable(pdf_path): #def extract_pdfcompositiontable(pdf_path):
    pdf = PDF(pdf_path)
    local_dfs = [] 

    # Extract tables using the correct function
    extracted_tables = pdf.extract_tables(ocr=ocr)

    # Get all tables from the single page (assuming page 0)
    extracted_tables = extracted_tables.get(0, [])  # Get tables from page 0 (default empty list if not found)
    
    for table in extracted_tables[1:]:
        df = table.df
        
        # Check if all column headers are integers
        if all(isinstance(col, int) for col in df.columns):
            # Drop the existing numeric header and set the first row as the new header
            df.columns = df.iloc[0]
            df = df[1:].reset_index(drop=True)
                
        # Replace "\n" in df
        df.columns = df.columns.str.replace("\n", " ")
        df = df.replace("\n", " ", regex=True)
        
        # Append the extracted DataFrame to the local list
        local_dfs.append(df)
    
    return local_dfs

### 7. Save list of dfs to a text file

In [ ]:
import os
import re

# Function to extract the numerical part of the filename
def extract_index(filename):
    # Use regular expression to find the numerical part of the filename
    match = re.search(r'\d+', filename)
    if match:
        return int(match.group())
    else:
        return None

# List to store DataFrames extracted from each pdf
dfs = []

# Sort pdf filenames based on their numerical part
pdf_filenames = sorted([filename for filename in os.listdir(pdf_folder) if filename.endswith(".pdf")], key=extract_index)

for pdf_filename in pdf_filenames:
    pdf_path = os.path.join(pdf_folder, pdf_filename)

    # Extract tables from the pdf
    tables = extract_pdfcompositiontable(pdf_path)
    batch_number = extract_batchnumber(pdf_path)

    if tables:
        for table in tables:

            # Insert the batch number as the first column of the DataFrame
            table.insert(0, 'Batch Number', batch_number)
            
            # Append the table to the list of DataFrames
            dfs.append(table.iloc[1:])

In [ ]:
len(dfs)

In [ ]:
dfs

In [ ]:
# Save dfs as text file for spaCy
def dfs_to_text(df_list, file_path):
    result_list = []
    for df in df_list:
        # Drop NaN columns
        df = df.dropna(axis=1, how='all')
        # Convert DataFrame to list of dictionaries
        data = [{"text": " ".join(str(val) for val in row.values)} for _, row in df.iterrows()]
        # Append data to result list
        result_list.extend(data)
        
        
    result_list = [result_list]

    # Write text data to file
    with open(file_path, "w") as outfile:
        for idx, item in enumerate(result_list):
            # Create a dictionary with "text" as key and the row as value
            outfile.write(str(item))
            if idx < len(result_list) - 1:
                outfile.write(",\n")
            else:
                outfile.write("\n")

    print(f"Text data written to {file_path}")

# Example usage
dfs_to_text(dfs, "dfs.txt")

#### 8. Keywords Matching Approach

In [ ]:
# Function to find and rename column names based on possible matches
def find_column_names(df, match_options):
    found_columns = {}
    for col in df.columns:
        if col is not None:  # Ensure column names are treated case-insensitively
            col_lower = col.lower()
            for key, options in match_options.items():
                if any(opt in col_lower for opt in options):
                    if key not in found_columns:  # Prevent duplicate keys
                        found_columns[key] = col
                        break
    return found_columns


# Standard columns all DataFrames should adhere to
standard_columns = ['Batch Number', 'Ingredient', 'Weight (%)', 'Weight(g)']

processed_dfs = []
for df in dfs:
    # Remove duplicate columns
    df = df.loc[:, ~df.columns.duplicated()]

    match_options = {
        'Batch Number': ['batch number'],
        'Ingredient': ['ingredient', 'ingrediant', 'name', 'Trade Name'],
        'Weight (%)': ['%', 'wt. percent', 'percent', '% w/w', '% wiw'],
        'Weight(g)': ['Batch Quantity', 'g', 'Batch (g)', 'Batch(g)', 'Weight(g)']
    }

    # Find actual columns based on match options
    actual_columns = find_column_names(df, match_options)

    # If matches are found, rename and reorder based on matches
    if actual_columns:
        df = df[list(actual_columns.values())].rename(columns={v: k for k, v in actual_columns.items()})
    else:
        # If no matches found, ensure the DataFrame still has the standard columns (all blank)
        df = pd.DataFrame(columns=standard_columns)
    
    # Ensure the DataFrame contains all standard columns, fill missing ones with None
    for col in standard_columns:
        if col not in df.columns:
            df[col] = None  # Add missing columns as None
    
    # Reorder columns to match the standard_columns order
    df = df[standard_columns]
    processed_dfs.append(df)

# Concatenate all processed DataFrames into a single final DataFrame
final_df = pd.concat(processed_dfs, ignore_index=True) if processed_dfs else pd.DataFrame(columns=standard_columns)

print(final_df) # including weight (g) as the last column

In [ ]:
# Delete the last col (weight (g)) - BMS doesn't need it in the end
df_3cols = final_df.iloc[:, :-1]
df_3cols

In [ ]:
# Post-prosseing with 'Weight %' column
# Assuming df_3cols is the dataframe you're working with

# Function to clean weight % values in the dataframe： 
# Since there are some percentage signs, I want to standardize the format.
def clean_weight(value):
    if pd.isna(value):
        return None  # If the value is NaN, return None
    if isinstance(value, str):  # Ensure the value is a string
        # Remove '%' and ',' characters
        value = value.replace('%', '').replace(',', '')
        # Keep only digits and decimal points
        value = ''.join(c for c in value if c.isdigit() or c == '.')
        try:
            return float(value)  # Convert to float
        except ValueError:
            return None  # In case the value cannot be converted to float, return None
    elif isinstance(value, (int, float)):
        return float(value)  # If it's already a numeric type, just convert to float
    else:
        return None  # If it's neither, return None

# Assuming df_3cols is your dataframe
df_3cols['Weight (%)'] = df_3cols['Weight (%)'].apply(clean_weight)

# Your dataframe is now cleaned: extracted the specific 3 columns successfully
print(df_3cols)


In [ ]:
# Saving the final DataFrame to CSV for accuracy part
df_3cols.to_csv('keywords_matching_dfs.csv', index=False) 
print(f"Saved {filename}")

#### 8a. Binary 0/1 Accuracy Metric for Keyword Matching

In [ ]:
df = df_3cols

# Convert 'Weight (%)' to string and remove rows where it contains '100' or '100%'
df = df[~df['Weight (%)'].astype(str).str.contains('^100(\.0+)?%?$', na=False)]

# Ensure that 'Weight (%)' is a string to replace '%' and convert to numeric
df['Weight (%)'] = df['Weight (%)'].astype(str).str.replace('%', '')
df['Weight (%)'] = pd.to_numeric(df['Weight (%)'], errors='coerce')

# Group by 'Batch Number' and sum 'Weight (%)', skipping NaN values automatically
sums = df.groupby('Batch Number')['Weight (%)'].sum()

# Create a dictionary to store the batch number and accuracy
batch_accuracy = {}

# Check sums and assign accuracy, with special handling for sums greater than 1
for batch_number, total_weight in sums.items():
    accuracy = 1 if total_weight == 100 else 0  # assuming the sum should be 100 for 100%
    batch_accuracy[batch_number] = accuracy
    if total_weight > 100:
        print(f"Sum greater than 100% for batch {batch_number} may be because of skipped dot '.' issue. [For example: 4.5 --> 45]")

# Create a new DataFrame from the dictionary
accuracy_df = pd.DataFrame(list(batch_accuracy.items()), columns=['Batch Number', 'Accuracy'])

# Display the new DataFrame
print(accuracy_df)

In [ ]:
# Calculate the accuracy percentage
accuracy_percentage = (accuracy_df['Accuracy'].sum() / len(accuracy_df)) * 100

print(f"Accuracy Percentage: {accuracy_percentage}%")

In [ ]:
# Calculate the accuracy percentage
accuracy_percentage = (accuracy_df['Accuracy'].sum() / len(accuracy_df)) * 100

print(f"Accuracy Percentage: {accuracy_percentage}%") 

#### Accuracy Post-processing for Keyword Matching

In [10]:
import pandas as pd

# Replace 'file_path.csv' with the path to your keywords matching CSV file
final_keywords_method_path = 'keywords_matching_dfs.csv'
df = pd.read_csv(final_keywords_method_path)

In [ ]:
# Convert 'Weight (%)' from string to numeric
df['Weight (%)'] = pd.to_numeric(df['Weight (%)'], errors='coerce')

# Remove rows where 'Weight (%)' is greater than or equal to 97
df = df[df['Weight (%)'] < 97] # because there are some total intra-granular rows (97,98.5...)

# Ensure that 'Weight (%)' is a string to replace '%' and convert to numeric
df['Weight (%)'] = df['Weight (%)'].astype(str).str.replace('%', '')
df['Weight (%)'] = pd.to_numeric(df['Weight (%)'], errors='coerce')

# Group by 'Batch Number' and sum 'Weight (%)', skipping NaN values automatically
sums = df.groupby('Batch Number')['Weight (%)'].sum()

# Create a dictionary to store the batch number and accuracy
batch_accuracy = {}

# Check sums and assign accuracy, with special handling for sums greater than 1
for batch_number, total_weight in sums.items():
    accuracy = 1 if total_weight == 100 or total_weight == 97 or abs(total_weight - 100) <= 0.03  else 0  # assuming the sum should be 100 for 100%, but there are three batch number have weight % sum as 97, 100.02.100.0224
    batch_accuracy[batch_number] = accuracy
    if total_weight > 100.02:
        print(f"Sum greater than 100.02% for batch {batch_number} may be because of skipped dot '.' issue or redacted ingredients.")# [For example: 4.5 --> 45 or duplicated rows])

# Create a new DataFrame from the dictionary
accuracy_df = pd.DataFrame(list(batch_accuracy.items()), columns=['Batch Number', 'Accuracy'])

# Display the new DataFrame
print(accuracy_df)

#### Strict 0 or 1 Accuracy for Keyword Matching

In [ ]:
# Calculate the accuracy percentage
accuracy_percentage = (accuracy_df['Accuracy'].sum() / len(accuracy_df)) * 100

print(f"Accuracy Percentage for Final Data Frame.csv: {accuracy_percentage}%")

#### 8b. Partial Accuracy for Keyword Matching

In [5]:
# Replace 'file_path.csv' with the path to your CSV file
final_keywords_method_path = 'keywords_matching_dfs.csv'
df = pd.read_csv(final_keywords_method_path)

In [ ]:
# Convert 'Weight (%)' from string to numeric
df['Weight (%)'] = pd.to_numeric(df['Weight (%)'], errors='coerce')

# Remove rows where 'Weight (%)' is greater than or equal to 94
df = df[df['Weight (%)'] < 94]  # Removed values >= 94

# Group by 'Batch Number' and sum 'Weight (%)', skipping NaN values automatically
sums = df.groupby('Batch Number')['Weight (%)'].sum()

# Create a dictionary to store the batch number and accuracy
batch_accuracy = {}

# Check sums and assign accuracy, with special handling for accuracy calculation
for batch_number, total_weight in sums.items():
    if total_weight == 100 or total_weight == 97 or abs(total_weight - 100) <= 0.03:
        accuracy = 1
    else:
        # Partial accuracy calculation for cases where the total weight is not close to 100
        accuracy = total_weight / 100

    # Store accuracy
    batch_accuracy[batch_number] = accuracy

# Create a new DataFrame from the dictionary
accuracy_df = pd.DataFrame(list(batch_accuracy.items()), columns=['Batch Number', 'Accuracy'])

# Display the new DataFrame
print(accuracy_df)

In [ ]:
# Calculate the accuracy percentage 
accuracy_percentage = ((accuracy_df['Accuracy'].sum()) / len(accuracy_df)) * 100

print(f"Accuracy Percentage for Final Data Frame.csv: {accuracy_percentage}%")

### 9. spaCy Usage

To test model:

In [ ]:
#read in txt file containing data
with open('dfs_txt.txt', 'r') as file:
    # Read the string from the file
    string_data = file.read()

# Use ast.literal_eval to convert the string to a Python object
data = ast.literal_eval(string_data)

In [ ]:
import pandas as pd
import spacy
from collections import defaultdict
import numpy as np


nlp = spacy.load("/path_to_model")

test_data = data 

# Initialize an empty list to store entity dictionaries
entity_data_list = []

# Process each text in the test data
for item in test_data:
    text = item["text"]
    doc = nlp(text)

    # Initialize a dictionary to store entities for this text
    entity_data = defaultdict(str)

    # Process each entity in the text
    for ent in doc.ents:
        # Use entity labels as keys in the dictionary
        label = ent.label_
        # Exclude "ner" label
        if label != "ner":
            # Add entity text to the corresponding key
            entity_data[label] += ent.text + " "

    # Append the entity data dictionary to the list
    entity_data_list.append(entity_data)

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(entity_data_list)

# Save the DataFrame to a CSV file
results_df.to_csv('ner_results.csv')


### 10. spaCy post processing

In [ ]:
#read in csv and rename column
df = pd.read_csv("ner_results.csv")
df = df.rename(columns={"PERCENT QUANTITY":"Weight (%)"})

In [ ]:
#process the weight column for rows that have repeated weights that are the same, if they are different, just keep since it is incorrect results
def process_weight(x):
    #split string
    parts = str(x).split(" ")
    #if there are multiple elements and first and second are the same, just keep the first weight
    if len(parts) > 1 and parts[0] == parts[1]:
        return parts[0]
    #if they are different, keep both since it is incorrect
    else:
        return x
    
df['Weight (%)'] = df['Weight (%)'].apply(process_weight)

In [ ]:
#tidy up nans
df["Weight (%)"] = df["Weight (%)"].replace('nan', np.nan)

In [ ]:
#drop column
df = df.drop(columns = ["Unnamed: 0"])

In [ ]:
#remove repeated ingredient names in a row (sometimes they are duplicated)
def remove_repeats(value):
    # Check if the value is a string
    if isinstance(value, str):
        # Use regex to find repeated substrings and replace them with a single instance
        return re.sub(r'(.+?)\1+', r'\1', value)
    else:
        return value
    
# Apply the function to the 'Column' and create a new cleaned column
df['INGREDIENT'] = df['INGREDIENT'].apply(remove_repeats)

In [ ]:
#sometimes, the weight % ends up mislabelled as an ingredient, post process
def move_number(row):
    weight = row['Weight (%)']
    ingredient = row['INGREDIENT']
    
    # Check if Weight (%) is NaN and INGREDIENT contains a number after the string
    if pd.isna(weight) and isinstance(ingredient, str):
        match = re.search(r'\d+(\.\d+)?', ingredient)  # Search for number in INGREDIENT
        if match:
            row['Weight (%)'] = float(match.group())  # Convert the matched number to float
            row['INGREDIENT'] = re.sub(r'\d+(\.\d+)?', '', ingredient).strip()  # Remove number from INGREDIENT
    return row

# Apply the function to each row
df = df.apply(move_number, axis=1)

In [ ]:
#remove any leading whitespace incase, and remove any percentages in drug loading column
df['BATCH NUMBER'] = df['BATCH NUMBER'].str.strip()
df['DRUG LOADING'] = df['DRUG LOADING'].str.replace('%', '')

In [ ]:
#for some batch records with inconsistent table formats, some rows might duplicate, so remove accordingly
df.drop_duplicates(subset=['BATCH NUMBER', 'Weight (%)', 'INGREDIENT', 'DRUG LOADING'], keep='first', inplace=True)

### 10a. Binary Accuracy for spaCy output

In [ ]:
df['Weight (%)'] = pd.to_numeric(df['Weight (%)'], errors='coerce')

# Remove rows where 'Weight (%)' is greater than or equal to 97
df = df[df['Weight (%)'] < 97]

# Ensure that 'Weight (%)' is a string to replace '%' and convert to numeric
df['Weight (%)'] = df['Weight (%)'].astype(str).str.replace('%', '')
df['Weight (%)'] = pd.to_numeric(df['Weight (%)'], errors='coerce')

# Group by 'Batch Number' and sum 'Weight (%)', skipping NaN values automatically
sums = df.groupby('BATCH NUMBER')['Weight (%)'].sum()

# Create a dictionary to store the batch number and accuracy
batch_accuracy = {}

# Check sums and assign accuracy, with special handling for sums greater than 1
for batch_number, total_weight in sums.items():
    accuracy = 1 if total_weight == 100 or total_weight == 97 or abs(total_weight - 100) <= 0.02  else 0  # assuming the sum should be 100 for 100%, but there are two batch number have weight % sum as 97, 100.02
    batch_accuracy[batch_number] = accuracy
    if total_weight > 100.02:
        print(f"Sum greater than 100.02% for batch {batch_number} may be because of skipped dot '.' issue. [For example: 4.5 --> 45]")

# Create a new DataFrame from the dictionary
accuracy_df = pd.DataFrame(list(batch_accuracy.items()), columns=['BATCH NUMBER', 'Accuracy'])

# Display the new DataFrame
print(accuracy_df)

accuracy_percentage = ((accuracy_df['Accuracy'].sum()) / len(accuracy_df)) * 100

print(f"Accuracy Percentage for Final Data Frame.csv: {accuracy_percentage}%")

### 10b. Partial Accuracy for spaCy output

In [ ]:
df['Weight (%)'] = pd.to_numeric(df['Weight (%)'], errors='coerce')

# Remove rows where 'Weight (%)' is greater than or equal to 97
df = df[df['Weight (%)'] < 97]  # Removed values >= 97

# Group by 'Batch Number' and sum 'Weight (%)', skipping NaN values automatically
sums = df.groupby('BATCH NUMBER')['Weight (%)'].sum()

# Create a dictionary to store the batch number and accuracy
batch_accuracy = {}

# Check sums and assign accuracy, with special handling for accuracy calculation
for batch_number, total_weight in sums.items():
    if total_weight == 100 or abs(total_weight - 100) <= 0.03:
        accuracy = 1
    else:
        # Partial accuracy calculation for cases where the total weight is not close to 100
        accuracy = total_weight / 100

    # Store accuracy
    batch_accuracy[batch_number] = accuracy

# Create a new DataFrame from the dictionary
accuracy_df = pd.DataFrame(list(batch_accuracy.items()), columns=['BATCH NUMBER', 'Accuracy'])

# Display the new DataFrame
print(accuracy_df)

accuracy_percentage = ((accuracy_df['Accuracy'].sum()) / len(accuracy_df)) * 100

print(f"Accuracy Percentage for Final Data Frame.csv: {accuracy_percentage}%")